In [1]:
# cross-equation coveriance error term
# in each case error terms are gaussian
# what are they picking up
# unobservable errors are correlated

import numpy as np
from matplotlib.pyplot import plot
import pandas as pd
from pathlib import Path
import yfinance as yf
from arch.univariate import ARCH, GARCH
from arch.univariate import ARX

local_drive = Path('C:/Users/torin/OneDrive')
databonds = Path('Desktop/Thesis/data/bonds/model')
datamacro = Path('Desktop/Thesis/data/macro')

def normalize_periods(df):
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.set_index(['DATE']).resample('MS').mean()
    return df

def calc_difference(df, col_names, new_col_name):
    df[new_col_name] = df[col_names[0]] - df[col_names[1]]
    return df

def calc_ratio(df, col_names, new_col_name):
    df[new_col_name] = df[col_names[0]]/df[col_names[1]]
    return df

def get_fx():
    eur = get_yf('EURUSD')[['Date','Close']]
    eur.rename({'Date':'DATE'}, axis = 1, inplace=True)
    eur['Close'] = pd.to_numeric(eur['Close'])
    eur['DATE'] = eur['DATE'].astype(str).apply(lambda x : x[0:10])
    eur = normalize_periods(eur)
    eur.columns = ['eur']

    brl = get_yf('BRLUSD')[['Date','Close']]
    brl.rename({'Date':'DATE'}, axis = 1, inplace=True)
    brl['Close'] = pd.to_numeric(brl['Close'])
    brl['DATE'] = brl['DATE'].astype(str).apply(lambda x : x[0:10])
    brl = normalize_periods(brl)
    brl.columns = ['brl']

    ch = get_yf('CLPUSD')[['Date','Close']]
    ch.rename({'Date':'DATE'}, axis = 1, inplace=True)
    ch['Close'] = pd.to_numeric(ch['Close'])
    ch['DATE'] = ch['DATE'].astype(str).apply(lambda x : x[0:10])
    ch = normalize_periods(ch)
    ch.columns = ['clp']
    ch['clp'] = 1/ch['clp']

    fx = pd.concat([eur, brl, ch], axis = 1)

    return fx

def get_yf(ticker):    
    yfpath = Path('C:/Users/torin/OneDrive/Desktop/Thesis/data/macro/yf')
    data = pd.read_csv(yfpath / str(ticker + '.csv'))
    return data

def get_yahoo_finance_series(ticker, name):
    data = yf.Ticker(ticker)
    hist = data.history(period="25y")
    hist = hist.reset_index()
    hist['Date'] = pd.to_datetime(hist['Date'])
    hist.to_csv("C:/Users/torin/OneDrive/Desktop/Thesis/data/macro/yf" +'/' + name + '.csv')
    return hist

def convert_to_usd(col_to_convert, original_currency, df):
    fx = get_fx().resample('MS').mean().to_dict()
    df['fx'] = df.index.map(fx[original_currency])
    df[col_to_convert] = df[col_to_convert]*df['fx']
    del df['fx']
    return df

tenors=[2,3,4,5,6,7,8,10,12,15,20,25,30]

monhtly_sector_spreads = {}
for tenor in tenors:
    temp = pd.read_csv(local_drive / databonds / Path('IT_BR') / Path('monthly_spreads_' + str(tenor) + '.csv'))
    temp.rename({'date':'DATE'}, axis =1, inplace=True)
    temp = normalize_periods(temp)
    monhtly_sector_spreads[tenor] = temp

In [9]:
########### macro variables ###############

    
def get_gdp(growth, lag):
    # EXOG VARIABLES
    # gd

    gdplevel = pd.read_csv(local_drive / datamacro / Path('model')  / Path('oecd_weekly_gdp_levels.csv') )
    gdppercapitalevel = pd.read_csv(local_drive / datamacro / Path('model')  / Path('oecd_weekly_gdp_level_percapita.csv') )

    del gdplevel['Unnamed: 0']
    del gdppercapitalevel['Unnamed: 0']

    gdplevel.rename({'date':'DATE'}, axis =1, inplace=True)
    gdppercapitalevel.rename({'date':'DATE'}, axis =1, inplace=True)

    gdplevel = gdplevel.pivot(index=['DATE'], values = ['GDP'], columns=['region']).reset_index()
    gdplevel.columns = ['DATE', 'Brazil_GDP','Chile_GDP',  'Italy_GDP', 'United States_GDP']

    gdppercapitalevel = gdppercapitalevel.pivot(index=['DATE'], values = ['GDP'], columns=['region']).reset_index()
    gdppercapitalevel.columns = ['DATE', 'Brazil_GDPcapita', 'Chile_GDPcapita','Italy_GDPcapita', 'United States_GDPcapita']

    gdplevel['DATE'], gdppercapitalevel['DATE'] = pd.to_datetime(gdplevel['DATE']), pd.to_datetime(gdppercapitalevel['DATE'] )

    gdplevel,gdppercapitalevel = normalize_periods(gdplevel), normalize_periods(gdppercapitalevel)

    if growth == True:
        for col in gdplevel:
            gdplevel[col] = (gdplevel[col]/(gdplevel[col].shift(12)) - 1)*100

        for col in gdppercapitalevel:
            gdppercapitalevel[col] = (gdppercapitalevel[col]/(gdppercapitalevel[col].shift(12)) - 1)*100

        gdplevel, gdppercapitalevel = gdplevel.dropna(), gdppercapitalevel.dropna()

    gdplevel,gdppercapitalevel  = calc_difference(gdplevel,['Brazil_GDP', 'Italy_GDP'], 'diff_br_it_gdp'), calc_difference(gdppercapitalevel,['Brazil_GDPcapita', 'Italy_GDPcapita'], 'diff_br_it_gdp_per_cp')
    gdplevel,gdppercapitalevel  = calc_difference(gdplevel,['Brazil_GDP', 'United States_GDP'], 'diff_br_us_gdp'), calc_difference(gdppercapitalevel,['Brazil_GDPcapita', 'United States_GDPcapita'], 'diff_br_us_gdp_per_cp')
    gdplevel,gdppercapitalevel  = calc_difference(gdplevel,['Italy_GDP', 'United States_GDP'], 'diff_it_us_gdp'), calc_difference(gdppercapitalevel,['Italy_GDPcapita', 'United States_GDPcapita'], 'diff_it_us_gdp_per_cp')
    gdplevel,gdppercapitalevel  = calc_difference(gdplevel,['Chile_GDP', 'United States_GDP'], 'diff_ch_us_gdp'), calc_difference(gdppercapitalevel,['Chile_GDPcapita', 'United States_GDPcapita'], 'diff_ch_us_gdp_per_cp')

    gdplevel,gdppercapitalevel  = calc_ratio(gdplevel,['Brazil_GDP', 'United States_GDP'], 'ratio_br_us_gdp'), calc_difference(gdppercapitalevel,['Brazil_GDPcapita', 'United States_GDPcapita'], 'ratio_br_us_gdp_per_cp')
    gdplevel,gdppercapitalevel  = calc_ratio(gdplevel,['Italy_GDP', 'United States_GDP'], 'ratio_it_us_gdp'), calc_difference(gdppercapitalevel,['Italy_GDPcapita', 'United States_GDPcapita'], 'ratio_it_us_gdp_per_cp')

    gdp = pd.concat([gdplevel, gdppercapitalevel ], axis = 1).dropna()

    if lag == True:
        for col in gdp.columns:
            gdp[col] = gdp[col].shift(3)
    
    gdp = gdp.dropna()

    return gdp

def get_inflation():
    # EXOG VARIABLES
    # inflation

    br_inflation = pd.read_csv(local_drive / datamacro / Path('model')  / Path('br_inflation.csv') )
    it_inflation = pd.read_csv(local_drive / datamacro / Path('model')  / Path('it_inflation.csv') )

    br_inflation['br_inflation_mom'] = br_inflation['IPCA'].pct_change()*100
    br_inflation['br_inflation_yoy'] = (br_inflation['IPCA'].div(br_inflation['IPCA'].shift(12)) - 1)*100

    del br_inflation['IPCA']
    br_inflation = br_inflation.dropna()


    it_inflation['it_inflation_mom'] = it_inflation['HICP - All Items'].pct_change()*100
    it_inflation['it_inflation_yoy'] = (it_inflation['HICP - All Items'].div(it_inflation['HICP - All Items'].shift(12)) - 1)*100
    del it_inflation['HICP - All Items']
    it_inflation.rename({'Date':'DATE'}, axis =1, inplace=True)
    it_inflation = it_inflation.dropna()

    it_inflation['DATE'], br_inflation['DATE'] = pd.to_datetime(it_inflation['DATE']), pd.to_datetime(br_inflation['DATE'] )

    it_inflation,br_inflation = normalize_periods(it_inflation), normalize_periods(br_inflation)

    inflation = pd.concat([br_inflation, it_inflation ], axis = 1).dropna()

    inflation = calc_difference(inflation, ['br_inflation_yoy', 'it_inflation_yoy'], 'diff_br_it_inflation_yoy')
    inflation = calc_difference(inflation, ['br_inflation_mom', 'it_inflation_mom'], 'diff_br_it_inflation_mom')

    return inflation

def get_debt(gdplevel):
    # EXOG VARIABLES
    # debt
    br_debt = pd.read_csv(local_drive / datamacro / Path('model')  / Path('br_debt.csv') )
    br_debt_nominal = br_debt.pivot(index=['DATE'], values = ['VALUE (R$)'], columns=['category']).reset_index().dropna(axis = 1, how = 'all')
    br_debt_nominal.columns = br_debt_nominal.columns.droplevel(0)
    br_debt_nominal.rename(columns={ br_debt_nominal.columns[0]: "DATE" }, inplace = True)

    br_debt_perc = br_debt.pivot(index=['DATE'], values = ['VALUE ((% PIB))'], columns=['category']).reset_index().dropna(axis = 1, how = 'all')
    br_debt_perc.columns = br_debt_perc.columns.droplevel(0)
    br_debt_perc.rename(columns={ br_debt_perc.columns[0]: "DATE" }, inplace = True)

    br_debt_nominal['DATE'], br_debt_perc['DATE'] = pd.to_datetime(br_debt_nominal['DATE']), pd.to_datetime(br_debt_perc['DATE'])
    br_debt_nominal, br_debt_perc = normalize_periods(br_debt_nominal),normalize_periods(br_debt_perc)

    br_debt_nominal = convert_to_usd('DLSP - Dívida Fiscal Líquida','brl',br_debt_nominal)
    br_debt_nominal = convert_to_usd('DLSP - dívida externa líquida - total','brl',br_debt_nominal)
    br_debt_nominal = convert_to_usd('DLSP - dívida interna líquida','brl',br_debt_nominal)


    it_debt = pd.read_csv(local_drive / datamacro/ Path('model') /Path('bank_of_italy_statistics.csv'))
    it_debt.rename({'Observation date':'DATE'}, axis =1, inplace=True)
    it_debt = it_debt[['DATE','General Government: gross debt']]
    it_debt['DATE'] = pd.to_datetime(it_debt['DATE'] )
    it_debt = normalize_periods(it_debt)
    it_debt = convert_to_usd('General Government: gross debt','eur',it_debt)

    debt = pd.concat([br_debt_nominal, br_debt_perc , it_debt], axis = 1).dropna()

    debt = debt[['DLSP - Dívida Fiscal Líquida', 'DLSP - Dívida Fiscal Líquida (% of pib)', 'General Government: gross debt']]

    debt['it_gdp_temp'] = pd.to_datetime(debt.index.copy()).map(gdplevel[['Italy_GDP']].to_dict()['Italy_GDP'])
    debt['it_debt_perc_gdp'] = (debt['General Government: gross debt']/debt['it_gdp_temp'])*100
    debt = debt.dropna()
    del debt['it_gdp_temp']
    debt = calc_difference(debt, ['DLSP - Dívida Fiscal Líquida (% of pib)', 'it_debt_perc_gdp'], 'diff_br_it_debt_perc_gdp')
    debt = calc_difference(debt, ['DLSP - Dívida Fiscal Líquida', 'General Government: gross debt'], 'diff_br_it_debt')
    return debt

def get_deficit(gdplevel):

    # EXOG VARIABLES
    # debt servicing and deficits

    it_deficit = pd.read_csv(local_drive / datamacro / Path('model')  / Path('it_deficit_trend.csv'))
    it_deficit.rename({'Observation date':'DATE'}, axis =1, inplace=True)

    br_deficit = pd.read_csv(local_drive / datamacro / Path('model')  / Path('br_deficit.csv'))

    deficit_code_map = {'BM12_NFGFJNNS12': 'NFSP - governo federal e Banco Central - juros nominais', 
    'BM12_NFGFNNAS12': 'NFSP - governo federal e Banco Central - nominal', 
    'BM12_NFGFNYS12': 'NFSP - governo federal e Banco Central - primário'}

    br_deficit['CODE'] = br_deficit['CODE'].map(deficit_code_map)

    br_deficit_nominal = br_deficit.pivot(index=['DATE'], values = ['VALUE (R$)'], columns=['CODE']).reset_index()
    br_deficit_nominal = br_deficit_nominal.dropna(how = 'all', axis = 1)
    br_deficit_nominal.columns = br_deficit_nominal.columns.droplevel(0)
    br_deficit_nominal.rename(columns={ br_deficit_nominal.columns[0]: "DATE" }, inplace = True)

    br_deficit_perc = br_deficit.pivot(index=['DATE'], values = ['VALUE ((% PIB))'], columns=['CODE']).reset_index()
    br_deficit_perc = br_deficit_perc.dropna(how = 'all', axis = 1)
    br_deficit_perc.columns = br_deficit_perc.columns.droplevel(0)
    br_deficit_perc.rename(columns={ br_deficit_perc.columns[0]: "DATE" }, inplace = True)

    br_deficit_nominal['DATE'], br_deficit_perc['DATE'], it_deficit['DATE'] = pd.to_datetime(br_deficit_nominal['DATE']), pd.to_datetime(br_deficit_perc['DATE']), pd.to_datetime(it_deficit['DATE'] )

    it_deficit, br_deficit_nominal, br_deficit_perc = normalize_periods(it_deficit),normalize_periods(br_deficit_nominal), normalize_periods(br_deficit_perc)

    br_deficit_nominal = convert_to_usd('NFSP - governo federal e Banco Central - nominal','brl',br_deficit_nominal)
    br_deficit_nominal = convert_to_usd('NFSP - governo federal e Banco Central - juros nominais','brl',br_deficit_nominal)
    it_deficit = convert_to_usd('trend - state deficit','eur',it_deficit)

    deficit = pd.concat([it_deficit, br_deficit_nominal,br_deficit_perc ], axis = 1).dropna()
    deficit['it_gdp_temp'] = pd.to_datetime(deficit.index.copy()).map(gdplevel[['Italy_GDP']].to_dict()['Italy_GDP'])
    deficit['it_deficit_perc_gdp'] = (deficit['trend - state deficit']/deficit['it_gdp_temp'])*100
    deficit.dropna()
    del deficit['it_gdp_temp']

    deficit = calc_difference(deficit, ['NFSP - governo federal e Banco Central - nominal', 'trend - state deficit'], 'diff_br_it_deficit')
    deficit = calc_difference(deficit, ['NFSP - governo federal e Banco Central - primário', 'it_deficit_perc_gdp'], 'diff_br_it_deficit_perc_gdp')

    return deficit

def get_foreign_reserves(gdplevel):
    br_fr = pd.read_csv(local_drive / datamacro / Path('model')  / Path('br_foreign_reserves.csv') )
    code_map = {'BM12_RES12': 'Foreign Reserves'}
    br_fr['CODE'] = br_fr['CODE'].map(code_map)
    br_fr = br_fr[['DATE','VALUE (US$)']]
    br_fr = normalize_periods(br_fr)
    br_fr.columns = ['br_foreign_reserves']

    br_fr['br_gdp_temp'] = pd.to_datetime(br_fr.index.copy()).map(gdplevel[['Brazil_GDP']].to_dict()['Brazil_GDP'])
    br_fr['br_foreign_reserves_gdp'] = (br_fr['br_foreign_reserves']/br_fr['br_gdp_temp'])*100
    del br_fr['br_gdp_temp']

    # Official reserve assets - Foreign currency reserves (in convertible foreign currencies)
    it_fr = pd.read_csv(local_drive / datamacro/ Path('model') /Path('bank_of_italy_statistics.csv'))
    it_fr.rename({'Observation date':'DATE'}, axis =1, inplace=True)
    it_fr = it_fr[['DATE','Official reserve assets - Foreign currency reserves (in convertible foreign currencies)']]
    it_fr = normalize_periods(it_fr)
    it_fr.columns = ['it_foreign_reserves']

    it_fr['it_gdp_temp'] = pd.to_datetime(it_fr.index.copy()).map(gdplevel[['Italy_GDP']].to_dict()['Italy_GDP'])
    it_fr['it_foreign_reserves_gdp'] = (it_fr['it_foreign_reserves']/it_fr['it_gdp_temp'])*100
    del it_fr['it_gdp_temp']

    fr = pd.concat([br_fr, it_fr], axis = 1)

    fr = calc_difference(fr, ['br_foreign_reserves','it_foreign_reserves'], 'diff_br_it_foreign_reserves')
    fr = calc_difference(fr, ['br_foreign_reserves_gdp','it_foreign_reserves_gdp'], 'diff_br_it_foreign_reserves_gdp')

    return fr

def get_fx():
    eur = get_yf('EURUSD')[['Date','Close']]
    eur.rename({'Date':'DATE'}, axis = 1, inplace=True)
    eur['Close'] = pd.to_numeric(eur['Close'])
    eur['DATE'] = eur['DATE'].astype(str).apply(lambda x : x[0:10])
    eur = normalize_periods(eur)
    eur.columns = ['eur']

    brl = get_yf('BRLUSD')[['Date','Close']]
    brl.rename({'Date':'DATE'}, axis = 1, inplace=True)
    brl['Close'] = pd.to_numeric(brl['Close'])
    brl['DATE'] = brl['DATE'].astype(str).apply(lambda x : x[0:10])
    brl = normalize_periods(brl)
    brl.columns = ['brl']

    ch = get_yf('CLPUSD')[['Date','Close']]
    ch.rename({'Date':'DATE'}, axis = 1, inplace=True)
    ch['Close'] = pd.to_numeric(ch['Close'])
    ch['DATE'] = ch['DATE'].astype(str).apply(lambda x : x[0:10])
    ch = normalize_periods(ch)
    ch.columns = ['clp']

    fx = pd.concat([eur, brl, ch], axis = 1)

    return fx


def get_balance_of_trade(gdplevel):

    balancetradebr = pd.read_csv(local_drive / datamacro/ Path('model') /Path('br_balancetrade.csv'))
    balancetradeit  = pd.read_csv(local_drive / datamacro/ Path('model') /Path('it_balance_of_trade.csv'))

    balancetradeit.rename({'Select time':'DATE'}, axis = 1, inplace=True)

    balancetradebr['DATE'] = pd.to_datetime(balancetradebr['DATE'] )
    balancetradeit['DATE'] = pd.to_datetime(balancetradeit['DATE'] )

    balancetradebr = balancetradebr.pivot(index = 'DATE', columns='Cat',values='VALUE (US$)').reset_index()

    balancetradebr = normalize_periods(balancetradebr)
    balancetradeit = normalize_periods(balancetradeit)

    balancetradeit = convert_to_usd('export - value (millions of euros)','eur',balancetradeit)
    balancetradeit = convert_to_usd('import - value (millions of euros)','eur',balancetradeit)
    balancetradeit = convert_to_usd('trade balance - value (millions of euros)','eur',balancetradeit)

    balancetradeit.rename({'export - value (millions of euros)':'it export - value (millions of dollars)'}, axis = 1, inplace=True)
    balancetradeit.rename({'import - value (millions of euros)':'it import - value (millions of dollars)'}, axis = 1, inplace=True)
    balancetradeit.rename({'trade balance - value (millions of euros)':'it trade balance - value (millions of dollars)'}, axis = 1, inplace=True)
    
    balancetrade = pd.concat([balancetradebr,balancetradeit], axis = 1)

    balancetrade['it_gdp_temp'] = pd.to_datetime(balancetrade.index.copy()).map(gdplevel[['Italy_GDP']].to_dict()['Italy_GDP'])
    balancetrade['it_trade_bal_gdp'] = (balancetrade['it trade balance - value (millions of dollars)']/balancetrade['it_gdp_temp'])*100
    balancetrade = balancetrade.dropna()
    del balancetrade['it_gdp_temp']

    balancetrade['br_gdp_temp'] = pd.to_datetime(balancetrade.index.copy()).map(gdplevel[['Brazil_GDP']].to_dict()['Brazil_GDP'])
    balancetrade['br_trade_bal_gdp'] = (balancetrade['balance_of_trade']/balancetrade['br_gdp_temp'])*100
    balancetrade = balancetrade.dropna()
    del balancetrade['br_gdp_temp']

    return balancetrade

def debt_composition():
    it = pd.read_csv(local_drive / datamacro/ Path('model') /Path('bank_of_italy_statistics.csv'))
    it.rename({'Observation date':'DATE'}, axis =1, inplace=True)
    
    it_debt_holders = ['General Government: gross debt held by other residents (share)',
       'General Government: gross debt held by other monetary financial institutions (share)',
       'General Government: gross debt held by other financial institutions (share)',
       'General Government: gross debt held by central bank (share)',
       'General Government: gross debt held by non-residents (share)',]
     
    it = it[['DATE'] + it_debt_holders]
    it = normalize_periods(it)
    it = it[['General Government: gross debt held by non-residents (share)',
             'General Government: gross debt held by other monetary financial institutions (share)', 
             'General Government: gross debt held by central bank (share)']]
    it.columns = ['it_non_resident_share', 'it_other_mon_share', 'it_cb_share']

    it['it_central_banks_share'] = it['it_other_mon_share'] + it['it_cb_share']

    br = pd.read_csv(local_drive / datamacro/ Path('model') /Path('br_debt_holders.csv'))
    br['DATE'] = br['DATE'].str.replace('/','-')
    br['DATE'] = pd.to_datetime(br['DATE'] )
    br = br.set_index(['DATE'])
    for col in br.columns:
        br[col] = pd.to_numeric(br[col].astype(str).str.replace(',',''))

    br = br.reset_index()
    br = normalize_periods(br)
    br = br[['Non-Resident Total (share)']]
    br.columns = ['br_non_resident_share']

    brcbshare = pd.read_csv((local_drive / datamacro/ Path('model') /Path('br_cb_share.csv')))
    brcbshare.columns = ['Date', 'br_cb_share']
    brcbshare['Date'] = pd.to_datetime(brcbshare['Date'])
    brcbshare = brcbshare.set_index(['Date']).to_dict()['br_cb_share']

    br['br_central_banks_share'] = br.index.map(brcbshare)

    non_res = pd.concat([br, it], axis = 1)

    non_res = calc_difference(non_res, ['br_non_resident_share', 'it_non_resident_share'], 'diff_non_resident_share')
    non_res = calc_difference(non_res, ['br_central_banks_share', 'it_central_banks_share'], 'diff_central_banks_share')

    return non_res.dropna()


def exchange_rate_vol(exchange_rate):

    for col in exchange_rate.columns:
        ar = ARX(100 * np.log(exchange_rate[col].dropna()), lags=[1, 3, 12])
        ar.volatility = ARCH(p=5)
        res = ar.fit(update_freq=0, disp="off")
        vol = pd.DataFrame(res.conditional_volatility.dropna())
        vol.columns = [col + '_vol']

        exchange_rate[col + '_vol'] = vol[col + '_vol']

    return exchange_rate

gdp = get_gdp(growth=False, lag = False)
gdp_growth = get_gdp(growth=True, lag = False)
gdp_growth = gdp_growth[['Brazil_GDP','Italy_GDP']]
gdp_growth.columns = ['brazil_gdp_growth','italy_gdp_growth']

inflation = get_inflation()
debt = get_debt(gdp)
deficit = get_deficit(gdp)
tradebal = get_balance_of_trade(gdp)
exchange_rate = get_fx()
exchange_rate = exchange_rate_vol(exchange_rate)
foreign_reserves = get_foreign_reserves(gdp)
non_residents = debt_composition()


C:\Users\torin\AppData\Local\Temp\ipykernel_12996\3675667053.py:284: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  br['DATE'] = pd.to_datetime(br['DATE'] )


In [10]:
########### external/market variables ###############

def get_vix():
    vix = get_yf('VIX')[['Date','Close']]
    vix.rename({'Date':'DATE'}, axis = 1, inplace=True)
    vix['Close'] = pd.to_numeric(vix['Close'])
    vix['DATE'] = vix['DATE'].astype(str).apply(lambda x : x[0:10])
    vix = normalize_periods(vix)
    vix.columns = ['vix']
    return vix

def get_us3m():
    us3m = get_yf('DTB3')[['Date','Close']]
    us3m.rename({'Date':'DATE'}, axis = 1, inplace=True)
    us3m['Close'] = pd.to_numeric(us3m['Close'], errors='coerce')
    us3m = us3m.dropna()
    us3m['DATE'] = us3m['DATE'].astype(str).apply(lambda x : x[0:10])
    us3m = normalize_periods(us3m)
    us3m.columns = ['us3m']
    return us3m

def get_gold():
    gold = get_yf('GOLD')[['Date','Close']]
    gold.rename({'Date':'DATE'}, axis = 1, inplace=True)
    gold['Close'] = pd.to_numeric(gold['Close'])
    gold['DATE'] = gold['DATE'].astype(str).apply(lambda x : x[0:10])
    gold = normalize_periods(gold)
    gold.columns = ['gold']
    return gold

def get_us_high_yield():
    ush = get_yf('US_HIGH_YIELD')[['DATE','BAMLH0A0HYM2EY']]
    ush.rename({'BAMLH0A0HYM2EY':'us_high_yield'}, axis = 1, inplace=True)
    ush['us_high_yield'] = pd.to_numeric(ush['us_high_yield'], errors='coerce')
    ush = ush.dropna()
    ush['DATE'] = ush['DATE'].astype(str).apply(lambda x : x[0:10])
    ush = normalize_periods(ush)
    ush.columns = ['us_high_yield']
    return ush

def get_commods():
    commods = get_yf('DBC')[['Date','Close']]
    commods.rename({'Date':'DATE'}, axis = 1, inplace=True)
    commods['Close'] = pd.to_numeric(commods['Close'])
    commods['DATE'] = commods['DATE'].astype(str).apply(lambda x : x[0:10])
    commods = normalize_periods(commods)
    commods.columns = ['commods']
    return commods

def get_fx():
    eur = get_yf('EURUSD')[['Date','Close']]
    eur.rename({'Date':'DATE'}, axis = 1, inplace=True)
    eur['Close'] = pd.to_numeric(eur['Close'])
    eur['DATE'] = eur['DATE'].astype(str).apply(lambda x : x[0:10])
    eur = normalize_periods(eur)
    eur.columns = ['eur']

    brl = get_yf('BRLUSD')[['Date','Close']]
    brl.rename({'Date':'DATE'}, axis = 1, inplace=True)
    brl['Close'] = pd.to_numeric(brl['Close'])
    brl['DATE'] = brl['DATE'].astype(str).apply(lambda x : x[0:10])
    brl = normalize_periods(brl)
    brl.columns = ['brl']

    ch = get_yf('CLPUSD')[['Date','Close']]
    ch.rename({'Date':'DATE'}, axis = 1, inplace=True)
    ch['Close'] = pd.to_numeric(ch['Close'])
    ch['DATE'] = ch['DATE'].astype(str).apply(lambda x : x[0:10])
    ch = normalize_periods(ch)
    ch.columns = ['clp']

    fx = pd.concat([eur, brl, ch], axis = 1)

    return fx

def get_contagion():

    itvix = pd.read_csv(local_drive / databonds / 'itvix.csv')
    brvix = pd.read_csv(local_drive / databonds/ 'brvix.csv')

    itvix = itvix[['Date', 'Price']]
    brvix = brvix[['Date', 'Price']]

    itvix.rename({'Date':'DATE'}, axis = 1, inplace=True)
    brvix.rename({'Date':'DATE'}, axis = 1, inplace=True)

    itvix['Price'] = pd.to_numeric(itvix['Price'].astype(str).str.replace(',',''))
    brvix['Price'] = pd.to_numeric(brvix['Price'].astype(str).str.replace(',',''))

    itvix = normalize_periods(itvix)
    brvix = normalize_periods(brvix)

    itvix.columns = ['it_contagion']
    brvix.columns = ['br_contagion']

    ar = ARX(brvix, lags=[3])
    ar.volatility = ARCH(p=5)
    res = ar.fit(update_freq=0, disp="off")
    vol = pd.DataFrame(res.conditional_volatility.dropna())
    vol.columns = ['br_contagion']

    brvix['br_contagion'] = vol['br_contagion']

    contagion = pd.concat([itvix, brvix], axis = 1)
    return contagion.dropna()

vix = get_vix()
ushighyield = get_us_high_yield()
gold = get_gold()
commods = get_commods()
us3m = get_us3m()
contagion = get_contagion()


C:\Users\torin\AppData\Local\Temp\ipykernel_12996\3004767102.py:19: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['DATE'] = pd.to_datetime(df['DATE'])
C:\Users\torin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 8.247e+04. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.1 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [11]:
########### merging final dataset ################

estimates = {}
for k,v in monhtly_sector_spreads.items():
    estimates[k] = pd.concat([gdp,
                              gdp_growth,
                              inflation, 
                              debt, 
                              deficit, 
                              exchange_rate,
                              foreign_reserves,
                              non_residents,
                              tradebal,
                              vix, 
                              ushighyield, 
                              gold, 
                              commods,
                              us3m,
                              contagion,
                              v], axis = 1).dropna()

def get_model_data(estimates, measure = 'spread'):

  estall = pd.concat(estimates).reset_index()
  del estall['level_0']
  estall['tenor'] = 'tenor_' + estall['tenor'].astype(str).apply(lambda x : str(x)[0:2].replace('.',''))
  
  if measure == 'spread':
    tenorsdf = estall.pivot(columns=['tenor'], values = ['spread_BR_IT'], index = ['DATE'])
    tenorsdf.columns = tenorsdf.columns.get_level_values(1)
    tenorsdf = tenorsdf.add_prefix('spread_BR_IT_')
  
  elif measure == 'yield_br':
    tenorsdf = estall.pivot(columns=['tenor'], values = ['yield BR'], index = ['DATE'])
    tenorsdf.columns = tenorsdf.columns.get_level_values(1)
    tenorsdf = tenorsdf.add_prefix('yield_br_')

  elif measure == 'yield_it':
    tenorsdf = estall.pivot(columns=['tenor'], values = ['yield IT'], index = ['DATE'])
    tenorsdf.columns = tenorsdf.columns.get_level_values(1)
    tenorsdf = tenorsdf.add_prefix('yield_it_')
  
  estall = estall.set_index(['DATE']).drop([ 
      'tenor', 
      'yield BR', 
      'yield IT',
      'spread_BR_IT'], axis = 1).drop_duplicates().sort_index()

  model_data = pd.concat([tenorsdf, 
                          estall], axis = 1)

  model_data = model_data.dropna()

  cols_rename = {'DLSP - Dívida Fiscal Líquida':'br_divida_fiscal_liquida',
        'DLSP - dívida externa líquida - total': 'br_divida_externa_liquida',
        'DLSP - dívida interna líquida': 'br_divida_interna_liquida',
        'DLSP - Dívida Fiscal Líquida (% of pib)': 'br_divida_fiscal_liquida_pct_pib',
        'DLSP - dívida externa líquida - total (% of pib)': 'br_divida_externa_liquida_pib',
        'DLSP - dívida interna líquida (% of pib)': 'br_divida_interna_liquida_pib',
        'General Government: gross debt':'it_gross_debt',
        'trend - state deficit': 'it_trend_state_deficit',
        'NFSP - governo federal e Banco Central - juros nominais': 'br_juros_nominais',
        'NFSP - governo federal e Banco Central - nominal': 'br_deficit_nominal',
        'NFSP - governo federal e Banco Central - primário': 'br_deficit_primario',
        'United States_GDP':'us_gdp',
        'United States_GDPcapita':'us_gdp_per_capita',
        'Brazil_GDP':'br_gdp',
        'Italy_GDP':'it_gdp', 
        'Italy_GDPcapita':'it_gdp_per_capita',
          'Brazil_GDPcapita':'br_gdp_per_capita'}

  model_data.rename(cols_rename, axis =1, inplace=True)

  return model_data

get_model_data(estimates).to_csv(local_drive / databonds/ 'model_data_spread_br_it.csv')
get_model_data(estimates, measure = 'yield_it').to_csv(local_drive / databonds/ 'model_data_yield_it.csv')
get_model_data(estimates, measure = 'yield_br').to_csv(local_drive / databonds/ 'model_data_yield_br.csv')

In [2]:

def get_data(measure = 'spread_br_it'):
    if measure == 'spread_br_it':
        data = pd.read_csv(local_drive / databonds/'model_data_spread_br_it.csv').set_index(['DATE'])
    if measure == 'yield_it':
        data = pd.read_csv(local_drive / databonds/'model_data_yield_it.csv').set_index(['DATE'])
    if measure == 'yield_br':
        data = pd.read_csv(local_drive / databonds/'model_data_yield_br.csv').set_index(['DATE'])
    return data